# Lesson 1: Data Sources & Formats

**Module 3: Data & Pipeline Engineering**  
**Estimated Time**: 3-4 hours  
**Difficulty**: Beginner-Intermediate

---

## 🎯 Learning Objectives

By the end of this lesson, you will:

✅ Understand the different types of data sources in production ML systems  
✅ Grasp the critical trade-offs between text vs binary and row vs column-major formats  
✅ Know when to use CSV, Parquet, Avro, and JSON  
✅ Benchmark read/write performance and compression strategies  
✅ Answer 5 interview questions on data sources and formats  

---

## 📚 Table of Contents

1. [The Data Landscape in Production ML](#1-data-landscape)
2. [Data Sources in Production Systems](#2-data-sources)
3. [Data Formats: Text vs Binary](#3-text-vs-binary)
4. [Data Formats: Row-Major vs Column-Major](#4-row-vs-column)
5. [Format Comparison Table](#5-format-comparison)
6. [Hands-On: Benchmarking Performance](#6-benchmarking)
7. [Compression Strategies](#7-compression)
8. [Column Pruning Demo](#8-column-pruning)
9. [Exercises](#9-exercises)
10. [Interview Preparation](#10-interview)

---

## 1. The Data Landscape in Production ML <a id='1-data-landscape'></a>

In machine learning operations (MLOps), success hinges not just on models but on the **data pipelines** that feed those models.

Production machine learning is a different beast entirely. Here, the cleverest model architecture is worthless if it’s fed unreliable data or if its predictions can’t be reproduced.

### The Fundamental Truth

> **In an enterprise MLOps setting, engineers operate under a fundamental truth: models are often commodities, but the data and the pipelines that process it are the durable, defensible assets that drive business value.**

The raw material of ML is data, and the choices we make about data have profound downstream consequences on:
- **Performance** – garbage in, garbage out
- **Scalability** – can the pipeline handle 10× growth?
- **Reliability** – will the system work at 3 AM on a holiday?
- **Reproducibility** – can we recreate yesterday’s training run?

### High-Level Data Flow in a Production ML System

```
  ┌──────────────┐   ┌──────────────┐   ┌──────────────┐   ┌──────────────┐
  │  DATA        │   │ DATA         │   │  FEATURE     │   │  MODEL       │
  │  SOURCES     │─▶│ PIPELINE     │─▶│  STORE       │─▶│  TRAINING    │
  │              │   │ (ETL/ELT)    │   │              │   │              │
  └──────────────┘   └──────────────┘   └──────────────┘   └──────────────┘
    ▲ User Input       ▲ Clean,              ▲ Consistent      ▲
    ▲ Logs            transform,          features for     Uses
    ▲ Databases       validate            train & serve    Parquet
    ▲ Third-Party
```

### Why This Matters for Interviews

At companies like **Uber, Airbnb, Netflix** — **60-80% of ML engineering work is data engineering**. You cannot claim to be a Senior Data Scientist without understanding the data infrastructure.

---

## 2. Data Sources in Production Systems <a id='2-data-sources'></a>

Data in a production environment is **not** a clean, static CSV file. It is a dynamic, messy, and continuous flow of signals from a multitude of sources, each with its own characteristics and requirements.

### Data Sources Overview

```
  ┌─────────────────────────────────────────────────────────┐
  │              DATA SOURCES FOR ML SYSTEMS                  │
  ├─────────────┬─────────────┬──────────────┬───────────────┤
  │ USER INPUT  │ SYSTEM LOGS │ INTERNAL DBs │ THIRD-PARTY    │
  │             │             │              │                │
  │ - Searches  │ - Events    │ - Inventory  │ - Demographics │
  │ - Uploads   │ - Metrics   │ - CRM        │ - Social Media │
  │ - Forms     │ - Errors    │ - Accounts   │ - Weather      │
  │ - Surveys   │ - Latency   │ - Finance    │ - Economics    │
  │             │             │              │                │
  │ Reliability:│ Reliability:│ Reliability: │ Reliability:   │
  │  LOW        │  MEDIUM     │  HIGH        │  VARIABLE      │
  └─────────────┴─────────────┴──────────────┴───────────────┘
```

### 2.1 User Input Data

This is data explicitly provided by users — text in a search bar, uploaded images, form submissions, etc.

**Key characteristics:**
- **Notoriously unreliable** — users are lazy. If it’s possible to input unformatted and raw data, they will.
- Requires **heavy-duty validation** and **robust error handling**
- Examples: search queries, uploaded documents, survey responses, user profiles

**Real-world example:** An e-commerce search bar receives queries like `"blue sheos"`, `"  NIKE Air max  "`, and `"shoes < $50"`. Your pipeline must handle misspellings, inconsistent casing, extra whitespace, and special characters.

### 2.2 System-Generated Data (Logs)

Applications and infrastructure generate a **massive volume of logs**. These record:
- Significant events
- System states (CPU/memory usage)
- Service calls
- Model predictions and latencies

**Key characteristics:**
- Often **noisy** but invaluable for debugging and monitoring
- Can be processed in **batches** (daily/weekly) or in **real-time** (for alerts)
- Usually stored as semi-structured JSON or plain text

**Real-world example:** A model serving endpoint logs every prediction: `{"timestamp": "2024-01-15T10:30:00Z", "model_version": "v2.3", "input_features": {...}, "prediction": 0.87, "latency_ms": 23}`

### 2.3 Internal Databases

This is where enterprises typically derive **most value** from. Databases managing:
- Inventory
- Customer relationships (CRM)
- User accounts
- Financial transactions

**Key characteristics:**
- Typically **highly structured** and follows a relational model
- Often the most valuable for **feature engineering**
- Subject to access controls and privacy regulations

**Real-world example:** A recommendation model might process a user’s query, but it must check an internal inventory database to ensure that the recommended products are **actually in stock** before displaying them.

### 2.4 Third-Party Data

Data acquired from external vendors:
- Demographic information
- Social media activity
- Purchasing habits
- Weather data, economic indicators

**Key characteristics:**
- Powerful for **bootstrapping models** (e.g., recommender systems cold-start)
- Availability is increasingly constrained by **privacy regulations** (GDPR, CCPA)
- May have different update frequencies and data quality standards

### Summary Table

| Source | Reliability | Volume | Structure | Example |
|--------|-----------|--------|-----------|----------|
| User Input | Low | Variable | Unstructured | Search queries, form data |
| System Logs | Medium | Very High | Semi-structured | Server logs, model predictions |
| Internal DBs | High | High | Structured | CRM, inventory, transactions |
| Third-Party | Variable | Variable | Mixed | Demographics, social media |

---

## 3. Data Formats: Text vs Binary <a id='3-text-vs-binary'></a>

The format you choose for storage is a **critical architectural decision** that directly impacts storage costs, access speed, and ease of use.

### Format Types Overview

```
                    DATA FORMATS
                        │
           ┌───────────┴───────────┐
           │                       │
      TEXT (Human)            BINARY (Machine)
           │                       │
     ┌─────┼─────┐         ┌─────┼─────┐
     │           │         │           │
    CSV        JSON      Parquet      Avro
  (tabular)  (nested)  (columnar)   (row-based)
```

### 3.1 Text Formats (JSON, CSV)

Text formats are **human-readable**. You can open a JSON or CSV file in a text editor and immediately understand its contents.

**Advantages:**
- Excellent for **debugging**, configuration, and data interchange
- JSON is ubiquitous due to its simplicity and flexibility
- Easy to inspect and validate manually

**Disadvantages:**
- **Verbose** and consume significantly more storage space
- Slower to parse (need to convert strings to native types)

### The Storage Cost Example

Consider storing the number `1000000`:
- **As text (CSV/JSON)**: 7 characters = **7 bytes** in ASCII
- **As binary (Parquet)**: 32-bit integer = **4 bytes**

That’s almost **2× the storage** for a single number! Now multiply this across billions of rows.

```
  Storing the integer 1000000:

  TEXT:   ['1'] ['0'] ['0'] ['0'] ['0'] ['0'] ['0']   = 7 bytes
  BINARY: [00000000 00001111 01000010 01000000]        = 4 bytes

  Saving: ~43%  →  At 1 billion rows, that’s GIGABYTES saved.
```

### 3.2 Binary Formats (Parquet, Avro)

Binary formats are **not human-readable** and are designed for machine consumption.

**Advantages:**
- Far more **compact** and efficient to process
- Schema is embedded (self-describing)
- Support efficient compression

**Disadvantages:**
- Cannot inspect manually (need special tools)
- A program must know the exact schema and layout to interpret the bytes

### Real-World Size Comparison

> A **14 MB CSV file** can be brought down to **6 MB** when converted to the binary Parquet format. For large-scale analytical workloads, binary formats like Parquet are the **industry standard**.

---

## 4. Data Formats: Row-Major vs Column-Major <a id='4-row-vs-column'></a>

This distinction is perhaps the **most critical** for an ML engineer to grasp, as it directly relates to how we typically access data for training and analysis.

### Memory Layout Comparison

```
  Original Table:
  ┌────┬──────┬─────┐
  │ ID │ Name │ Age │
  ├────┼──────┼─────┤
  │  1 │ John │  30 │
  │  2 │ Jane │  25 │
  │  3 │ Bob  │  35 │
  └────┴──────┴─────┘

  ROW-MAJOR (CSV, most databases):
  Memory: [1,John,30] [2,Jane,25] [3,Bob,35]
          └── Row 1 ─┘ └── Row 2 ─┘ └─ Row 3 ─┘
  → Good for: Reading entire rows (OLTP, point lookups)

  COLUMN-MAJOR (Parquet, Pandas DataFrame):
  Memory: [1,2,3] [John,Jane,Bob] [30,25,35]
          └─ IDs ─┘ └─── Names ───┘ └─ Ages ─┘
  → Good for: Reading specific columns (OLAP, ML, analytics)
```

### 4.1 Row-Major Format (CSV, most databases)

Consecutive elements of a **row** are stored next to each other.

**Optimized for:**
- **Write-heavy workloads** — adding new, complete records
- Retrieving **entire samples** (e.g., fetching all data for a specific user ID)
- Good for **OLTP** (Online Transaction Processing) systems

### 4.2 Column-Major Format (Parquet, Pandas DataFrame)

Consecutive elements of a **column** are stored next to each other.

**Optimized for:**
- **Analytical queries** — common in ML (e.g., `SELECT AVG(age) FROM users`)
- **Column pruning** — read only the columns you need
- **Compression** — similar values in a column compress better
- Good for **OLAP** (Online Analytical Processing) systems

### Why Column-Major Matters for ML

When training an ML model, you typically:
1. Select specific **features** (columns), not entire rows
2. Compute **statistics** per column (mean, std for normalization)
3. Apply **transformations** column-wise

Column-major format allows reading that one column as a **single, contiguous block of memory** — extremely efficient and cache-friendly.

### The ~20× Performance Difference

```
  Column-major iteration (Pandas):
  ┌───┬───┬───┬───┬───┐   Contiguous in memory
  │ 30│25│35│28│41│   → CPU cache HITS ⚡
  └───┴───┴───┴───┴───┘   ~2 microseconds per column

  Row-major iteration:
  ┌───┐     ┌───┐     ┌───┐   Scattered in memory
  │ 30│ ... │ 25│ ... │ 35│   → CPU cache MISSES 🐢
  └───┘     └───┘     └───┘   ~38 microseconds per row

  Result: ~20× slower for row-wise access!
```

The popular `pandas` library is built around the **column-major DataFrame**. Iterating row-by-row vs column-by-column reveals a dramatic performance gap:

- Iterating **32M+ rows by column**: ~2 microseconds
- Iterating **32M+ rows by row**: ~38 microseconds
- That’s a **~20× difference**!

This happens because:
- Individual columns may be spread across different memory locations
- But elements **within each column** are ALWAYS contiguous
- Processors are much more efficient with **contiguous blocks of memory** (cache hits vs cache misses)
- When iterating rows, the processor must jump from one memory location to another for each row element (cache misses)

> This isn’t a flaw in pandas; it’s a **direct consequence** of its underlying column-major data model.

---

In [1]:
# Demonstration: Row vs Column iteration speed in Pandas
import pandas as pd
import numpy as np
import time

# Create a large DataFrame
n_rows = 1_000_000
df = pd.DataFrame({
    'col_a': np.random.randn(n_rows),
    'col_b': np.random.randn(n_rows),
    'col_c': np.random.randn(n_rows),
    'col_d': np.random.randn(n_rows),
})

# Column-wise access (fast - contiguous memory)
start = time.time()
for col in df.columns:
    _ = df[col].sum()
col_time = time.time() - start

# Row-wise access (slow - non-contiguous memory)
start = time.time()
for idx in range(min(10000, n_rows)):  # Only 10K rows to keep demo fast
    _ = df.iloc[idx]
row_time = time.time() - start

print(f"Column-wise sum (all {n_rows:,} rows, 4 cols): {col_time:.4f}s")
print(f"Row-wise access (10,000 rows):                  {row_time:.4f}s")
print(f"\nRow access is ~{row_time/col_time:.0f}x slower (even on 100x fewer iterations!)")
print("\n>>> Takeaway: NEVER iterate rows in Pandas. Use vectorized operations.")

Column-wise sum (all 1,000,000 rows, 4 cols): 0.0213s
Row-wise access (10,000 rows):                  0.1760s

Row access is ~8x slower (even on 100x fewer iterations!)

>>> Takeaway: NEVER iterate rows in Pandas. Use vectorized operations.


## 5. Format Comparison Table <a id='5-format-comparison'></a>

| Feature | CSV | Parquet | Avro | JSON |
|---------|-----|---------|------|------|
| **Type** | Text (Human Readable) | Binary (Columnar) | Binary (Row-based) | Text (Human Readable) |
| **Schema** | None (Inferred) | Embedded in footer | JSON Schema in header | None (Self-describing) |
| **Orientation** | Row-major | Column-major | Row-major | Document-oriented |
| **Use Case** | Small data, Excel | Analytics, ML training | Streaming (Kafka) | APIs, config, logs |
| **Compression** | Poor | Excellent (Snappy/Gzip) | Good | Poor |
| **Write Speed** | Slow | Slow (encoding overhead) | Fast | Moderate |
| **Read Speed** | Very Slow | Very Fast (column pruning) | Fast | Slow (parsing) |
| **Schema Evolution** | None | Limited | Excellent | None |
| **Splittable** | Yes | Yes | Yes | Line-delimited only |
| **Ecosystem** | Universal | Spark, Pandas, Arrow | Kafka, Hadoop | Universal |

### Quick Decision Guide

```
  What format should I use?

  ML Training on tabular data?    ──▶  PARQUET  (always)
  Streaming pipeline (Kafka)?      ──▶  AVRO  (schema evolution + row writes)
  API data interchange?            ──▶  JSON  (universal, flexible)
  Quick exploration / non-tech?    ──▶  CSV   (but convert to Parquet later)
```

---

## 6. Hands-On: Benchmarking Performance <a id='6-benchmarking'></a>

Let's generate a large dataset and compare **CSV vs Parquet** on three dimensions:
1. **Write speed**
2. **Read speed**
3. **File size**

In [2]:
import pandas as pd
import numpy as np
import time
import os

# ============================================================
# Step 1: Generate a "Large" synthetic dataset (1 Million rows)
# ============================================================
print("Generating synthetic dataset with 1M rows...")
n_rows = 1_000_000

df = pd.DataFrame({
    'user_id': np.arange(n_rows),
    'category': np.random.choice(['electronics', 'clothing', 'food', 'books', 'sports'], n_rows),
    'price': np.round(np.random.uniform(1, 500, n_rows), 2),
    'quantity': np.random.randint(1, 20, n_rows),
    'rating': np.round(np.random.uniform(1, 5, n_rows), 1),
    'is_returned': np.random.choice([True, False], n_rows, p=[0.05, 0.95]),
    'timestamp': pd.date_range('2023-01-01', periods=n_rows, freq='s')
})

print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / (1024**2):.1f} MB")
df.head()

Generating synthetic dataset with 1M rows...
Dataset shape: (1000000, 7)
Memory usage: 88.5 MB


,user_id,category,price,quantity,rating,is_returned,timestamp
0,0,books,103.96,10,3.9,False,2023-01-01 00:00:00
1,1,clothing,326.23,2,1.8,False,2023-01-01 00:00:01
2,2,clothing,124.50,1,2.2,False,2023-01-01 00:00:02
3,3,books,152.62,2,2.9,False,2023-01-01 00:00:03
4,4,food,101.91,11,1.7,False,2023-01-01 00:00:04


In [3]:
# ============================================================
# Step 2: Benchmark CSV
# ============================================================
print("Benchmarking CSV...")

# Write
start = time.time()
df.to_csv('benchmark_data.csv', index=False)
write_csv = time.time() - start

# Read
start = time.time()
df_csv = pd.read_csv('benchmark_data.csv')
read_csv = time.time() - start

# File size
size_csv = os.path.getsize('benchmark_data.csv') / (1024 * 1024)

print(f"  Write: {write_csv:.2f}s | Read: {read_csv:.2f}s | Size: {size_csv:.1f} MB")

Benchmarking CSV...
  Write: 13.07s | Read: 3.58s | Size: 52.3 MB


In [4]:
# ============================================================
# Step 3: Benchmark Parquet (Snappy compression - default)
# ============================================================
print("Benchmarking Parquet (Snappy)...")

# Write
start = time.time()
df.to_parquet('benchmark_data.parquet', index=False, compression='snappy')
write_pq = time.time() - start

# Read
start = time.time()
df_pq = pd.read_parquet('benchmark_data.parquet')
read_pq = time.time() - start

# File size
size_pq = os.path.getsize('benchmark_data.parquet') / (1024 * 1024)

print(f"  Write: {write_pq:.2f}s | Read: {read_pq:.2f}s | Size: {size_pq:.1f} MB")

Benchmarking Parquet (Snappy)...
  Write: 1.86s | Read: 0.87s | Size: 14.2 MB


In [5]:
# ============================================================
# Step 4: Compare Results
# ============================================================
print("\n" + "="*60)
print("BENCHMARK RESULTS: CSV vs Parquet (1M rows)")
print("="*60)
print(f"{'Metric':<20} {'CSV':<15} {'Parquet':<15} {'Winner':<15}")
print("-"*60)
print(f"{'Write Time (s)':<20} {write_csv:<15.2f} {write_pq:<15.2f} {'Parquet' if write_pq < write_csv else 'CSV'}")
print(f"{'Read Time (s)':<20} {read_csv:<15.2f} {read_pq:<15.2f} {'Parquet' if read_pq < read_csv else 'CSV'}")
print(f"{'File Size (MB)':<20} {size_csv:<15.1f} {size_pq:<15.1f} {'Parquet' if size_pq < size_csv else 'CSV'}")
print("="*60)
print(f"\n📊 Parquet is {size_csv/size_pq:.1f}x smaller and {read_csv/read_pq:.1f}x faster to read!")
print("\n>>> For ML workloads with tabular data, ALWAYS prefer Parquet.")


BENCHMARK RESULTS: CSV vs Parquet (1M rows)
Metric               CSV             Parquet         Winner         
------------------------------------------------------------
Write Time (s)       13.07           1.86            Parquet
Read Time (s)        3.58            0.87            Parquet
File Size (MB)       52.3            14.2            Parquet

📊 Parquet is 3.7x smaller and 4.1x faster to read!

>>> For ML workloads with tabular data, ALWAYS prefer Parquet.


## 7. Compression Strategies <a id='7-compression'></a>

Parquet supports multiple compression codecs. The two most common are:

| Codec | Speed | Compression Ratio | Use Case |
|-------|-------|-------------------|----------|
| **Snappy** | ⚡ Very Fast | Moderate | Default. Best for interactive/Spark workloads |
| **Gzip** | 🐢 Slower | High | Best for archival / cold storage |
| **Zstd** | ⚡ Fast | High | Best of both worlds (newer) |
| **LZ4** | ⚡⚡ Fastest | Low-Moderate | When speed is everything |

### Rule of Thumb

```
  Hot data (frequently accessed)  ──▶  Snappy or LZ4
  Cold data (archival)            ──▶  Gzip or Zstd
  Spark workloads                 ──▶  Snappy (default for good reason)
```

In [6]:
# ============================================================
# Compression Comparison: Snappy vs Gzip vs None
# ============================================================
results = {}

for codec in ['snappy', 'gzip', None]:
    label = codec if codec else 'none'
    fname = f'benchmark_{label}.parquet'
    
    # Write
    start = time.time()
    df.to_parquet(fname, index=False, compression=codec)
    w_time = time.time() - start
    
    # Read
    start = time.time()
    pd.read_parquet(fname)
    r_time = time.time() - start
    
    size = os.path.getsize(fname) / (1024 * 1024)
    results[label] = {'write': w_time, 'read': r_time, 'size': size}
    os.remove(fname)

print("\n" + "="*65)
print("COMPRESSION COMPARISON (Parquet, 1M rows)")
print("="*65)
print(f"{'Codec':<12} {'Write(s)':<12} {'Read(s)':<12} {'Size(MB)':<12}")
print("-"*65)
for codec, r in results.items():
    print(f"{codec:<12} {r['write']:<12.3f} {r['read']:<12.3f} {r['size']:<12.1f}")
print("\n>>> Snappy: best balance. Gzip: smallest files but slower. None: fastest but largest.")


COMPRESSION COMPARISON (Parquet, 1M rows)
Codec        Write(s)     Read(s)      Size(MB)    
-----------------------------------------------------------------
snappy       0.480        0.162        14.2        
gzip         14.655       0.173        9.3         
none         0.544        0.276        19.9        

>>> Snappy: best balance. Gzip: smallest files but slower. None: fastest but largest.


## 8. Column Pruning Demo <a id='8-column-pruning'></a>

One of Parquet’s killer features is **column pruning**: reading ONLY the columns you need.

```
  CSV: Must read ALL columns, then filter
  ┌─────┬──────┬─────┬────┬──────┬─────┬─────────┐
  │ uid │ cat  │price│qty │rating│ ret │timestamp │  ← Read ALL (wasteful)
  └─────┴──────┴─────┴────┴──────┴─────┴─────────┘

  Parquet: Read ONLY requested columns
  ┌─────┬──────┐
  │price│rating│  ← Read ONLY 2 columns (fast!)
  └─────┴──────┘
  (other 5 columns never touch disk)
```

In CSV, you must read the **entire file** even if you only need 2 columns out of 100. With Parquet, you read precisely the columns requested — the rest are never loaded from disk.

In [7]:
# ============================================================
# Column Pruning: Read only specific columns
# ============================================================

# Save both formats
df.to_csv('benchmark_data.csv', index=False)
df.to_parquet('benchmark_data.parquet', index=False)

# --- CSV: Must read ALL columns, then select ---
start = time.time()
df_csv_subset = pd.read_csv('benchmark_data.csv', usecols=['price', 'rating'])
csv_prune_time = time.time() - start

# --- Parquet: Reads ONLY the requested columns from disk ---
start = time.time()
df_pq_subset = pd.read_parquet('benchmark_data.parquet', columns=['price', 'rating'])
pq_prune_time = time.time() - start

print(f"Reading 2 out of 7 columns:")
print(f"  CSV (usecols):     {csv_prune_time:.3f}s")
print(f"  Parquet (columns): {pq_prune_time:.3f}s")
print(f"  Speedup:           {csv_prune_time/pq_prune_time:.1f}x faster with Parquet")
print(f"\n>>> This gap grows MASSIVELY with more columns (imagine 500 columns, reading 10)")

# Cleanup
os.remove('benchmark_data.csv')
os.remove('benchmark_data.parquet')

Reading 2 out of 7 columns:
  CSV (usecols):     0.634s
  Parquet (columns): 0.041s
  Speedup:           15.5x faster with Parquet

>>> This gap grows MASSIVELY with more columns (imagine 500 columns, reading 10)


## 9. Exercises <a id='9-exercises'></a>

### Exercise 1: Schema Impact
Create a DataFrame with mixed types (int, float, string, datetime). Save it as CSV and read it back. Compare the dtypes before and after. What happens? Why is this a problem for ML pipelines?

### Exercise 2: JSON Benchmarking  
Extend the benchmarking code to include **JSON Lines** format (`df.to_json('data.jsonl', orient='records', lines=True)`). How does it compare to CSV and Parquet?

### Exercise 3: Real-World Scenario
You’re building an ML pipeline that:
- Receives 10M user events per day as JSON from an API
- Needs to store 6 months of history for training
- Training reads only 5 out of 30 columns

Write pseudocode for how you’d architect the storage layer. What format for ingestion? What format for storage? Why?

---

## 10. Interview Preparation <a id='10-interview'></a>

### Q1: "You need to store 1TB of tabular data for ML training. CSV or Parquet? Why?"

**Answer:**  
"Parquet, without question. Four key reasons:
1. **Columnar Storage**: ML training typically selects specific features (columns). Parquet allows column pruning — reading only the needed columns, saving enormous I/O.
2. **Compression**: Parquet with Snappy compression can reduce 1TB to ~200-300GB. Columnar data compresses far better because similar values are stored together.
3. **Schema Enforcement**: Parquet embeds the schema (dtypes), preventing ‘everything is a string’ issues common in CSV that cause silent ML bugs.
4. **Splittable**: Compressed Parquet files can be processed in parallel by Spark executors, enabling distributed training.

I’d use CSV only for small, human-inspectable files or when sharing data with non-technical stakeholders."

---

### Q2: "Explain the difference between row-major and column-major storage. When would you choose each?"

**Answer:**  
"Row-major stores consecutive elements of each row together (CSV, relational DBs). Column-major stores consecutive elements of each column together (Parquet, Pandas DataFrames).

Row-major excels at:
- **OLTP workloads**: Inserting/updating individual records
- **Point lookups**: Fetching all data for one user

Column-major excels at:
- **OLAP/ML workloads**: Computing statistics across millions of records
- **Feature selection**: Reading only needed columns from disk
- **Compression**: Similar values in a column compress very well

For ML, column-major is almost always preferred because we read many rows but few columns. The ~20× performance difference in Pandas column vs row iteration demonstrates this concretely."

---

### Q3: "When would you use Avro instead of Parquet?"

**Answer:**  
"I’d prioritize Avro for **streaming data pipelines**, especially with Kafka. Key reasons:
1. **Row-based**: Efficient for writing records one at a time (append-heavy)
2. **Schema evolution**: Avro handles adding/removing fields gracefully, which is crucial for long-running producers/consumers where schemas evolve over time
3. **Compact serialization**: Good for message passing between microservices

My typical pattern: ingest with Avro (streaming) → land in a data lake → convert to Parquet (analytics/training)."

---

### Q4: "Explain Dictionary Encoding in Parquet and why it matters for ML data."

**Answer:**  
"For categorical columns with low cardinality (few unique values), Parquet stores a dictionary of values (e.g., `['electronics', 'clothing', 'food']`) and replaces row values with tiny integer indices (e.g., `0, 1, 2`).

This provides:
1. **Massive compression** for categorical features common in ML datasets
2. **Faster filtering** since comparisons happen on small integers
3. **Reduced memory footprint** when loading into memory

This is especially impactful in ML where categorical features like `country`, `device_type`, `product_category` appear in nearly every dataset."

---

### Q5: "How would you design a data storage strategy for a system with both real-time serving and batch training needs?"

**Answer:**  
"I’d use a **Lambda Architecture** approach:

1. **Ingestion Layer**: Events arrive as JSON/Avro via Kafka (real-time)
2. **Speed Layer** (serving): Store in a low-latency database (Redis, DynamoDB) for real-time feature serving
3. **Batch Layer** (training): Land raw events in a data lake (S3/GCS) as Parquet files, partitioned by date
4. **Serving Layer**: Pre-computed features for model serving

For ML specifically:
- Training reads from Parquet in the batch layer (column pruning, compression)
- Serving reads from the speed layer (low-latency lookups)
- A feature store (like Feast) bridges both, ensuring training-serving consistency."

---

## 🎓 Key Takeaways

1. **Data is not a clean CSV** — production data comes from user input, logs, databases, and third parties, each with different reliability and structure
2. **Binary > Text** for large-scale processing (Parquet over CSV, always)
3. **Column-major > Row-major** for ML workloads (feature selection, statistics)
4. **Parquet is the gold standard** for ML training data storage
5. **Snappy compression** is the default for good reason (fast + decent ratio)
6. **Column pruning** provides massive I/O savings when you only need a few features

---

➡️ **Next Lesson**: [Lesson 2: ETL/ELT Pipelines](./lesson_02_etl_pipelines.ipynb) — Learn how to build production data pipelines.